### 컨볼루션 핵심 원리
| 단계  | 설명                   |
| --- | -------------------- |
| 1️ | 커널(필터)을 이미지 위에 덮는다   |
| 2️ | 덮인 영역의 픽셀과 커널 값을 곱한다 |
| 3️ | 다 더해서 새로운 픽셀 값을 만든다  |
| 4️ | 한 칸 이동해서 반복한다        |

### 커널 효과
| 효과      | 특징            | 대표 커널          |
| ------- | ------------- | -------------- |
| **스무딩** | 경계와 노이즈를 부드럽게 | 평균 필터, 가우시안 필터 |
| **엠보싱** | 경계 강조 + 양각 효과 | 방향성 있는 대칭X 필터  |

 ----
| 커널      | 효과               |
| ------- | ---------------- |
| 평균 필터   | 블러 (주변 평균)       |
| 샤프닝     | 경계 강조            |
| 가우시안 필터 | 노이즈 완화           |
| 소벨 필터   | 엣지 검출 (가로/세로 경계) |

In [4]:
import cv2 as cv
import numpy as np

# 1) 이미지 읽고 크기 줄이기
img = cv.imread('soccer.jpg')                             # 이미지 불러오기
img = cv.resize(img, dsize=(0,0), fx=0.4, fy=0.4)         # 가로세로 40%로 축소

# 2) 그레이스케일 변환 + 텍스트 추가
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)                # 컬러 → 흑백으로 변환
cv.putText(gray, 'soccer', (10, 20),                      # 좌상단에 텍스트 출력
           cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
cv.imshow('gray', gray)                                   # 변환된 흑백 이미지 띄우기

# 3) 가우시안 블러(스무딩) 강도 비교
# 커널 크기가 커질수록 더 많이 흐려짐
smooth = np.hstack((
    cv.GaussianBlur(gray, (5, 5), 0.0),                  # 5x5 블러
    cv.GaussianBlur(gray, (9, 9), 0.0),                  # 9x9 블러
    cv.GaussianBlur(gray, (15, 15), 0.0)                 # 15x15 블러 (가장 흐림)
))
cv.imshow('smooth', smooth)                              # 세 결과를 가로로 붙여 비교

# 4) 엠보싱 필터 커널 정의 (대각선 방향 경계 강조)
femboss = np.array([
    [-1.0, 0.0, 0.0],
    [ 0.0, 0.0, 0.0],
    [ 0.0, 0.0, 1.0]
])

# 5) 엠보싱 효과 적용 (정상 버전)
# 음수값 생기므로 int16으로 변환해야 정확함!
gray16 = np.int16(gray)                                  # uint8 → int16로 변환 (음수 처리 위해)
# 컨볼루션 + 128로 중앙값 맞춤 → clip으로 0~255 범위 제한 → 다시 uint8로
emboss = np.uint8(
    np.clip(cv.filter2D(gray16, -1, femboss) + 128, 0, 255)
)
cv.imshow('emboss', emboss)                              # 올바른 엠보싱 출력

# 6) 잘못된 예시 1: +128 없이 바로 clip (중심이 회색이 아니라 왜곡됨)
emboss_bad = np.uint8(
    np.clip(cv.filter2D(gray16, -1, femboss), 0, 255)
)
cv.imshow('emboss_bad', emboss_bad)                      # 중간값 보정 안 해서 양각 느낌이 안남

# 7) 잘못된 예시 2: int16 변환 없이 uint8 그대로 → 오버플로우 발생
emboss_worse = cv.filter2D(gray, -1, femboss)            # uint8 그대로 → 음수 안 됨 → 경계 이상함
cv.imshow('emboss_worse', emboss_worse)                  # 오버플로우로 깨짐

# 8) 모든 창 닫기
cv.waitKey(0)
cv.destroyAllWindows()
